In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
import json
import requests

In [18]:
splits = {
    "vista": "https://raw.githubusercontent.com/Project-MONAI/VISTA/main/vista3d/data/jsons/TotalSegmentatorV2_5_folds.json",
    "sat": "https://raw.githubusercontent.com/zhaoziheng/SAT-DS/main/data/split_json/TotalSegmentator.json"
} 

In [19]:
for k,v in splits.items():
    split = json.loads(requests.get(v).text)

    if k == "vista":
        train = split['training']
        test = split['testing']
        train = [dict["image"].split("/")[0] for dict in train]
        test = [dict["image"].split("/")[0] for dict in test]
        version = "v2"

    elif k == "sat":
        train = split['train']
        test = split['test']
        version = "v1"

    # Path to meta.csv file
    df = pd.read_csv(f"/mnt/data1/TotalSegmentator/{version}/processed/meta.csv")

    val_ratio = sum(df["split"] == "val")/sum(df["split"] == "train")
    train, val = train_test_split(train, test_size=val_ratio, random_state=42)

    df.loc[df["image_id"].isin(test), f"{k}_split"] = "test"
    df.loc[df["image_id"].isin(train), f"{k}_split"] = "train"
    df.loc[df["image_id"].isin(val), f"{k}_split"] = "val"

    print(f"Train: {len(train)}, Val: {len(val)}, Test: {len(test)} for {k} split")

    df.to_csv(f"/mnt/data1/TotalSegmentator/{version}/processed/meta.csv", index=False)


Train: 928, Val: 52, Test: 248 for vista split
Train: 911, Val: 51, Test: 240 for sat split


: 